In [1]:
import string
import random
import os 
import subprocess
from subprocess import Popen, PIPE, STDOUT
import sys
import json, sys
import multiprocessing
import time
import sys

#Paths
path_initial = os.getcwd()
print ("Inital path: {0}".format(path_initial))

#Random suffix for tmp filenames
letters = string.ascii_letters
random_suffix = ''.join(random.choice(letters) for i in range(10))
path_exp="/tmp/openwsn-"+random_suffix
os.mkdir(path_exp)
print("Temporary path: {0} ".format(path_exp))

Inital path: /home/theoleyre/openwsn/notebook
Temporary path: /tmp/openwsn-OsnDOpYQhi 


In [2]:
# Metadata for experiments
iotlab_user="theoleyr"
iotlab_exp_name="owsn-" + random_suffix 
iotlab_exp_duration="180"
iotlab_resume_exp=True

# Parameters of the experiment
exp_board="iot-lab_M3"
exp_toolchain="armgcc"
exp_archi="m3"
exp_site="grenoble"
exp_nodes_list=[ 213 , 223 ]
exp_dagroots_list=[ 204 ]

#openvisualizer configuration
exp_logging_conf=path_initial + "/loggers/logging.conf"

#Only in simulation mode!
exp_nbnodes="5"
exp_topology="---load-topology " + path_initial + "/topologies/topology-3nodes.json"
#code (git repositories)
code_sw_src=path_initial+"/../openvisualizer/"
code_sw_gitversion="e039a05"
code_fw_src=path_initial+"/../openwsn-fw/"
code_fw_gitversion="515eafa7"
code_fw_bin=code_fw_src+"build/iot-lab_M3_armgcc/projects/common/03oos_openwsn_prog"

print("All vairables initialized")

All vairables initialized


In [3]:
#tools

#Run an extern command
def run_command(cmd, path):
    process = Popen(cmd, shell=True, stdin=None, stdout=PIPE, stderr=STDOUT, close_fds=True, cwd=path)

    #print the stdout asynchronously
    for line in process.stdout:
        print(line.rstrip("\n"))
  
    if (process.wait() != 0):
        print("... Failure")
        exit(-5)
    else:
        print("... finished")
        
#keep all the threads
threads = list()

print("Tools terminated")

Tools terminated


In [4]:
#Compilation

cmd="scons board=" + exp_board + " toolchain=" + exp_toolchain + " " 
cmd=cmd +" boardopt=printf modules=coap,udp apps=cjoin,cexample debugopt=CCA,schedule "
cmd=cmd + " scheduleopt=anycast,lowestrankfirst "
badmaxrssi="100"
goodminrssi="100"
cmd=cmd + " stackcfg=badmaxrssi:"+badmaxrssi+",goodminrssi:"+goodminrssi + " "
cmd=cmd + " oos_openwsn "
#process = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True, cwd=code_fw_src)
#output = process.stdout.read()
#print(output)
sys.stdout.write('Compilation.....')
sys.stdout.flush()
run_command(cmd, code_fw_src)

Compilation.....scons: Reading SConscript files ...

 ___                 _ _ _  ___  _ _ 
| . | ___  ___ ._ _ | | | |/ __>| \ |
| | || . \/ ._>| ' || | | |\__ \|   |
`___'|  _/\___.|_|_||__/_/ <___/|_\_|
     |_|                  openwsn.org

scons: done reading SConscript files.
scons: Building targets ...
arm-none-eabi-size --format=berkeley -d --totals build/iot-lab_M3_armgcc/projects/common/03oos_openwsn_prog
   text	   data	    bss	    dec	    hex	filename
 146348	    208	  65328	 211884	  33bac	build/iot-lab_M3_armgcc/projects/common/03oos_openwsn_prog
 146348	    208	  65328	 211884	  33bac	(TOTALS)
scons: done building targets.
... finished


In [5]:
# Let us resume an experiment (if one exists)
if (iotlab_resume_exp == True):
    cmd = 'iotlab-experiment get -e'
    process = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)
    output = process.stdout.read()
    infos = json.loads(output)

# pick the last (most recent) experiment
try:
    exp_id_running=infos["Running"][-1]
    
    #Site identification (if the experiment is already running)
    cmd="iotlab-experiment get -i " + str(exp_id_running) + " -n"
    process = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)
    output = process.stdout.read()
    
    infos=json.loads(output)
    exp_site=infos["items"][0]["site"]
    print("The site of this already running experiment has been identified to {0}".format(exp_site))

    #nodes identification 
    print("Running nodes:")
    for node in infos["items"]:
        print("  -> {0}".format(node["network_address"]))
    print("Be careful: this list must be equal to:")
    print("  -> dagroot(s): {0}".format(exp_dagroots_list))
    print("  -> node(s): {0}".format(exp_nodes_list))

#no experiment has been found -> let us start a novel one
except:
    print("Start an experiment")
    exp_id_running=0
    cmd= "iotlab-experiment submit " + " -n "+iotlab_exp_name + " -d "+ iotlab_exp_duration + " -l "+ exp_site + "," + exp_archi + ","
    for i in range(len(exp_dagroots_list)):    
        if ( i != 0 ):
            cmd=cmd+"+"    
        cmd= cmd + str(exp_dagroots_list[i])
    for node in exp_nodes_list :
        cmd= cmd + "+" + str(node)
    
    print(cmd)
    process = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)
    output = process.stdout.read()
    print(output)
    infos=json.loads(output)
    exp_id_running=infos["id"]    

#id of the experiment (alreay running or just started)
print("Experiment id running: {0}".format(exp_id_running))     

The site of this already running experiment has been identified to grenoble
Running nodes:
  -> m3-213.grenoble.iot-lab.info
  -> m3-223.grenoble.iot-lab.info
  -> m3-204.grenoble.iot-lab.info
Be careful: this list must be equal to:
  -> dagroot(s): [204]
  -> node(s): [213, 223]
Experiment id running: 262988


In [6]:
# waiting for the running mode
cmd="iotlab-experiment wait -i "+ str(exp_id_running)
process = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)
output = process.stdout.read()
print(output)    

Waiting that experiment 262988 gets in state Running
"Running"



In [7]:
#Flashing the devices
print("Flashing motes...")
cmd="iotlab-node --flash " + code_fw_bin + " -i " + str(exp_id_running)
print(cmd)
process = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)
output = process.stdout.read()
infos=json.loads(output)
ok=True
if "0" in infos:
    for info in infos["0"]:
        print("{0}: ok".format(info))

if "1" in infos:
    for info in infos["1"]:
        print("{0}: ko".format(info))
        ok = False
if ( ok == False ):
    print("Some motes have not been flashed correctly, stop now")
    exit(6)

print(".. terminated")

Flashing motes...
iotlab-node --flash /home/theoleyre/openwsn/notebook/../openwsn-fw/build/iot-lab_M3_armgcc/projects/common/03oos_openwsn_prog -i 262988
m3-204.grenoble.iot-lab.info: ok
m3-213.grenoble.iot-lab.info: ok
m3-223.grenoble.iot-lab.info: ok
.. terminated


In [8]:
#Install openvisualizer
print("Install the current version of Openvisualizer")
cmd="sudo pip install -e ."
process = Popen(cmd, shell=True, stdin=None, stdout=None, stderr=PIPE, close_fds=True, cwd=code_sw_src)
output = process.stderr.read()
print(output)
if (process.wait() != 0):
    print("Installation of openvisualizer has failed")
    exit(-7)
else:
    print("Installation ok")

Install the current version of Openvisualizer
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.

Installation ok


In [9]:
#construct the config file
file=open(exp_logging_conf, 'w')
print("Configuration file ....")

try:
    os.mkdir(path_exp+"/logs/")
except :
    print("Directory {0} already exists".format(path_exp+"/logs/"))
    
# constant beginning
file_start=open(exp_logging_conf+".start", 'r')
for line in file_start:
    file.write(line)
file_start.close()

file.write("[handler_std]\n")
file.write("class=logging.FileHandler\n")
file.write("args=('"+path_exp+"/logs/openv-server.log', 'w')\n")
file.write("formatter=std\n\n")

file.write("[handler_errors]\n")
file.write("class=logging.FileHandler\n")
file.write("args=('"+path_exp+"/logs/openv-server-errors.log', 'w')\n")
file.write("level=ERROR\n")
file.write("formatter=std\n\n")

file.write("[handler_success]\n")
file.write("class=logging.FileHandler\n")
file.write("args=('"+path_exp+"/logs/openv-server-success.log', 'w')\n")
file.write("level=SUCCESS\n")
file.write("formatter=std\n\n")

file.write("[handler_info]\n")
file.write("class=logging.FileHandler\n")
file.write("args=('"+path_exp+"/logs/openv-server-info.log', 'w')\n")
file.write("level=INFO\n")
file.write("formatter=std\n\n")

file.write("[handler_all]\n")
file.write("class=logging.FileHandler\n")
file.write("args=('"+path_exp+"/logs/openv-server-all.log', 'w')\n")
file.write("formatter=std\n\n")

file.write("[handler_html]\n")
file.write("class=logging.FileHandler\n")
file.write("args=('"+path_exp+"/logs/openv-server-all.html.log', 'w')\n")
file.write("formatter=console\n\n")


#constant end
file_end=open(exp_logging_conf+".end", 'r')
for line in file_end:
    file.write(line)
file_end.close()

#end of the config file
file.close()

print("... finished")

Configuration file ....
... finished


In [10]:
#construct the command with all the options for openvisualizer
openvisualizer_options="--opentun --wireshark-debug --mqtt-broker 127.0.0.1 -d --fw-path /home/theoleyre/openwsn/openwsn-fw"
openvisualizer_options=openvisualizer_options+ " --lconf " + exp_logging_conf
if ( exp_board == "iot-lab_M3" ):
    cmd="sudo openv-server " + openvisualizer_options + " --iotlab-motes "       
    for i in range(len(exp_dagroots_list)):    
        cmd=cmd + exp_archi + "-" + str(exp_dagroots_list[i]) + "." + exp_site + ".iot-lab.info "
    for i in range(len(exp_nodes_list)):    
        cmd=cmd + exp_archi + "-" + str(exp_nodes_list[i]) + "." + exp_site + ".iot-lab.info "
    print(cmd)
elif ( exp_board == "python" ):
    cmd="sudo openv-server " + openvisualizer_options + " --sim "+ exp_nb_nodes + " " + exp_topology

# stops the previous process
try:
    print(process_openvisualizer)
    process_openvisualizer.terminate()    
except:
    print("No running openvisualizer process")
    
#Running the OV application
print("Running openvisualizer in a separated process") 
process_openvisualizer = multiprocessing.Process(target=run_command, args=(cmd, code_sw_src, ))
process_openvisualizer.start()
print("Process {0} started, pid {1}".format(process_openvisualizer, os.getpid()))


sudo openv-server --opentun --wireshark-debug --mqtt-broker 127.0.0.1 -d --fw-path /home/theoleyre/openwsn/openwsn-fw --lconf /home/theoleyre/openwsn/notebook/loggers/logging.conf --iotlab-motes m3-204.grenoble.iot-lab.info m3-213.grenoble.iot-lab.info m3-223.grenoble.iot-lab.info 
No running openvisualizer process
Running openvisualizer in a separated process
Process <Process(Process-1, started)> started, pid 7965
17:04:28 INFO opening tun interface
ioctl(TUNSETIFF): Device or resource busy
17:04:28 INFO created following virtual interfaces
6: tun0: <POINTOPOINT,MULTICAST,NOARP,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UNKNOWN group default qlen 500
    link/none 
    inet6 bbbb::1/64 scope global 
       valid_lft forever preferred_lft forever
    inet6 fe80::1/64 scope link 
       valid_lft forever preferred_lft forever
    inet6 fe80::a73b:e543:20f:85f9/64 scope link stable-privacy 
       valid_lft forever preferred_lft forever
17:04:29 SUCCESS m3-204.grenoble.iot-lab.info Ok.
1

In [ ]:
#wait that openvizualizer is properly initiated
cmd="openv-client motes"
while True:
    process = Popen(cmd, shell=True, stdin=None, stdout=PIPE, stderr=STDOUT, close_fds=True)
    output = process.stdout.read()
    if "Connection refused" not in output:
        break
    #wait 2 seconds before trying to connect to the server
    time.sleep(2)
print("Openvisualizer seems correctly running")

In [ ]:
#reboot all the motes (if some have been already selected dagroot for an unkwnon reason)
if ( exp_board == "iot-lab_M3" ):
    for node in exp_dagroots_list:
        cmd="openv-client root m3-" + str(node) +  "." + exp_site + ".iot-lab.info"
        print(cmd)
        process = Popen(cmd, shell=True, stdin=None, stdout=PIPE, stderr=STDOUT, close_fds=True)

        output = process.stdout.read()
        print(output)


In [ ]:
# Configuration: dagroot
if ( exp_board == "iot-lab_M3" ):
    for node in exp_dagroots_list:
        cmd="openv-client root m3-" + str(node) +  "." + exp_site + ".iot-lab.info"
        print(cmd)
        process = Popen(cmd, shell=True, stdin=None, stdout=PIPE, stderr=STDOUT, close_fds=True)

        output = process.stdout.read()
        print(output)
        if "Ok"  not in output:
            print("Error when setting up the dagroot {0}".format(node))

In [ ]:
#start the web client part 
cmd="openv-client view web --debug ERROR"
print(cmd)
process = Popen(cmd, shell=True, stdin=None, stdout=PIPE, stderr=STDOUT, close_fds=True)

#print the stdout asynchronously
for line in process.stdout:
    print(line.rstrip("\n"))

if (process.wait() != 0):
    print("... Failure")
    exit(-5)
else:
    print("... finished")

In [ ]:
print("Experiment terminated")